# README

Neurogenic bladder dysfunction caused by SCI leads to uncontrolled urination or inability to empty bladder, which could pose a threat on patients’ lives. Recent studies on animals showed that stimulation on spinal cord (SCS) could improve the ability of bladder control. While parallels between the effects on bladder function of more established techniques like electrical stimulation of the pudendal nerve exists,  the mechanisms by which SCS interacts with spinal circuits controlling the bladder is still unclear. 

In this case, computational models simulating epidural electrical stimulation on pudendo-vesical reflex represent an interesting tool to study the parallels between SCS and peripheral nerve stimulation of specific bladder afferent and efferent fibers. This repository contains the code of a computational model of bladder spinal reflexes for peripheral nerve stimulation. Several components of the model are constructed based on the previous study[1][2]:  

## Dependency installation

We suggest users to create a virtual environment or use docker as a container of the model.

The following packages are needed to successfully compile this model:

* python ~= 3.6
  * mpi4py ~= 3.0.3
  * Neuron ~= 7.8
  * numpy ~= 1.19.1
  * pandas ~= 1.1.1
  * matplotlib ~= 3.3.1
  * scipy ~= 1.5.0
  * argparse ~= 1.4.0
  * gxx_linux-64
* [openmpi-4.1.1](https://www.open-mpi.org/software/ompi/v4.1/)
* [neuron ~= 7.8.1.1](http://www.neuron.yale.edu/neuron/download)

Specifically, since NEURON is embedded in JupyterLab NEURON, there is no need to install or compile the application from NEURON website. However, the neuron package is still needed when the model is running on other instances.


A conda environment named 
```bladder_model_env.yaml```
is already setup for usage. Users can load the environment configuration file to create an environment. Open a terminal from the launcher, run the following commands in terminal to create an environment.


In [ ]:
# run in terminal
cd work

conda env create -f bladder_model_env.yaml

conda will start to fetch andn install all the packages needed. Unfortunately users need to repeat this step every time to create the environment, since when user exit the pipeline, all the environment settings will be cleared.

After successful installation of all packages, run the following steps to activate the environment

In [ ]:
# 1. run:
conda init 

# 2. you should be able to see the (base) in front of terminal prompt. if not, quit and restart the terminal

# 3. make sure you already have (base) in front of terminal prompt, run
conda activate bladder_model 

# 4. you should be able to see (bladder_model) in front of the terminal prompt. To terminate the environment, run
conda deactivate

## NEURON configuration

The folder 

```
neuralnetwork/code/mod_files 
```

contains MOD files representing different mechanisms that describe the membrane dynamics and particular cell properties. They need to be compiled and link to NEURON object for recognition.

In terminal, run


In [ ]:
# run in terminal
cd neuralnetwork/code/

nrnivmodl ./mod_files

Good news: these MOD files are not like conda environment. Once configured, users can skip this step in the future since the configured files will remain in the folder even after exiting the pipeline. 

**So far we have finished the environment setting.**


## Simulation preparations

There are 2 options for running simulations, but option 2 is really used in this pipeline for osparc users. See appendix for the different parameter requirements for these two options.

option 1: Users directly define the recruitment percentage of pudendal afferent, pelvic afferent and SPN to receive external stimulation.

**option 2:**  The model recieves recruitment data generated by upstream models and an EES amplitude. The recruitment percentages of afferents are calculated automatically based on the recruitment ratio.
            
For option 2, run the following code to fetch the predefined parameters.



After getting the recruitment data from upstream models, users need to define which recruitment data to use as the input for this model. 

Please go to directory /inputs, follow the instructions in `recruitment data splitter.ipynb`, and **COME BACK** to this README file for the last few steps of running the simulation on osparc. 


## Running a simulation (Back from `recruitment data splitter.ipynb`)

So far we have finished all the preparation work and we are ready for running a simulation
The `runBladderSim.py` is the entrance for running the simulation. Users can start the simulation by running this file in the terminal with specified parameters, or make some customized adjustment in `runBladderSim.py` as per the needs.

The program takes the following arguments:

+ inputFile: name of file which stores the neural network structure, can be found under /neuralnetwork/nnStructures/
+ time: the total simulation time in ms
+ rounds: the number of repetitions 
+ eesFrequency: the epidural electrical stimulaiton frequency applied on afferent nerves
+ eesAmplitude: the epidural electrical stimulaiton amplitude applied on afferent nerves
+ Pud: the ratio of Pudendal nerves stimulated, ranging from 0.0 - 1.0
+ Pel: the ratio of Pelvic nerves stimulated, ranging from 0.0 - 1.0
+ SPN: the ratio of SPN neurons stimulated, ranging from 0.0 - 1.0
+ instance number: the index of the kernel that running the model
+ start volume: the bladder volume at the start of the simulation
+ end volume: the bladder volume at the end of the simulation


After finishing all the preparation work, users should already have a command line using the following parameters generated from configurations:
+ time
+ eesFrequency
+ eesAmplitude
+ start volume
+ end volume

In the terminal, cd to /code, paste the command line you have and hit enter to start the simulation.

You can also compare the command line with the following example to see if you have a correct format. 

In [ ]:
# Example

cd neuralnetwork/code/

python scripts/runBladderSim.py frwSimCat.txt 10000 1 33 500 0.0 0.0 0.0 1 20.0 20.0


Pudendal afferent, Pelvic afferent and SPN recruitment rates are set to 0.0, and the 300 nA EES is applied on them. The recruitment ratio is calculated by the recruitment data generated by bladder model

## Results
All results are stored in ../results. There is also an analysis script `Analysis on bladder traces and neuron firings.ipynb` to provide some simple processing of data.

The types of data stored after simulation are listed below.

### Bladder pressure related
+ bladder pressure traces: 
    `instance-index_time_bp_volume_stim-freq.txt`, store the bladder pressur changes along the simulation
    
+ bladder pressure change ratio: 
    `instance-index_time_bp_pud-ratio_pel-ratio_spn-ratio at stim-freq.txt `, store 3 values: average bladder pressure pre-stimulation, average bladder pressure post-stimulation, post/pre bladder pressure ratio.
    
A ratio less than 1 indicates inhibition effect of EES, and a ratio larger than 1 indicates excitorary effect.
    
+ bladder pressure auc change ratio: 
    `instance-index_time_auc_pud-ratio_pel-ratio_spn-ratio at stim-freq.txt `, similar to the file setting above, store auc under the bladder pressure curve along simulation. Using auc as a more comparable measurement for ramp-filling experiment setting. 



### Neuron firing related 

+ afferents and neuron spikes: stored the timepoint for each neuron within the group (30 neurons in total) to spike in sparse matrix as .npz files

+ membrane potential: the membrane potential change for each SPN neuron within the group (30 neurons in total)  is stored as `instance-index_time_bladder-volume_stim-frequency_SPN_mem.txt`




## Reference

[1]: Formento, E. et al., Formento, E., Minassian, K., Wagner, F. *et al.* Electrical spinal cord stimulation must preserve proprioception to enable locomotion in humans with spinal cord injury. *Nat Neurosci* **21,** 1728–1741 (2018). https://doi.org/10.1038/s41593-018-0262-6          Related code: https://github.com/FormentoEmanuele/MuscleSpindleCircuitsModel

[2]:McGee MJ, Grill WM. Modeling the spinal pudendo-vesical reflex for bladder control by pudendal afferent stimulation. J Comput Neurosci. 2016 Jun;40(3):283-96. doi: 10.1007/s10827-016-0597-5. Epub 2016 Mar 11. PMID: 26968615; PMCID: PMC4860361.

## Appendix

### parameter requirements for option 1 and 2
| OPTION | Network structure file | EES frequency | EES amplitude | Recruitment rate of pudendal nerve ( 0.0 - 1.0) | Recruitment rate of pelvic nerve (0.0 - 1.0) |Recruitment rate of SPN neuron (0.0 - 1.0) |instance number|start volume|end volume |
| ------ | ---------------| --------- | ------ | ------------------------- | -------------------- |--------------------------- |---------------|------|----|                           
| 1      |     **✓**     |     **✓**     | any value     |     **✓**      |     **✓**    |     **✓**    |    **✓**       |     **✓**      |    **✓**    |               
| 2      |     **✓**     |     **✓**     |     **✓**     | 0.0            |     0.0      |       0.0     |    **✓**       |      **✓**     |    **✓**    |     


+ option 1 example: customize the recruitment rate:

```
  cd neuralnetwork/code/
  python scripts/runBladderSim.py frwSimCat.txt 10000 1 33 300 1.0 0.5 0.2 1 20.0 20.0
```

Pud recruitment is set to 1.0, while Pel recruitment is set to 0.5.
Notice the *eesAmplitude* (300) here works as a placeholder and will not be processed by the program. It could be any number.